In [2]:
import numpy as np
from nbeats_keras.model import NBeatsNet
import pandas as pd
from collections import deque

Using TensorFlow backend.


# Let's first predict NYC COVID confirmed cases for 1 day in advance

# data preprocessing for COVID cases in NYC,USA

In [3]:
df = pd.read_csv('https://query.data.world/s/gwkznsfzafx65bybjhytj3qlnf4eyc')
df.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Case_Type,People_Total_Tested_Count,Cases,Difference,Date,Combined_Key,Country_Region,Province_State,Admin2,iso2,iso3,FIPS,Lat,Long,Population_Count,People_Hospitalized_Cumulative_Count,Data_Source,Prep_Flow_Runtime
0,Confirmed,NaN,0,0,2/3/2020,Switzerland,Switzerland,NaN,NaN,CH,CHE,NaN,46.818200,8.227500,8654618.0,NaN,2019 Novel Coronavirus COVID-19 (2019-nCoV) Da...,5/16/2020 11:14:13 PM
1,Confirmed,NaN,23,0,4/21/2020,Antigua and Barbuda,Antigua and Barbuda,NaN,NaN,AG,ATG,NaN,17.060800,-61.796400,97928.0,NaN,2019 Novel Coronavirus COVID-19 (2019-nCoV) Da...,5/16/2020 11:14:13 PM
2,Deaths,NaN,0,0,3/1/2020,Cyprus,Cyprus,NaN,NaN,CY,CYP,NaN,35.126400,33.429900,1207361.0,NaN,2019 Novel Coronavirus COVID-19 (2019-nCoV) Da...,5/16/2020 11:14:13 PM
3,Deaths,NaN,56,0,5/11/2020,Thailand,Thailand,NaN,NaN,TH,THA,NaN,15.870032,100.992541,69799978.0,NaN,2019 Novel Coronavirus COVID-19 (2019-nCoV) Da...,5/16/2020 11:14:13 PM
4,Deaths,NaN,0,0,2/11/2020,Jamaica,Jamaica,NaN,NaN,JM,JAM,NaN,18.109600,-77.297500,2961161.0,NaN,2019 Novel Coronavirus COVID-19 (2019-nCoV) Da...,5/16/2020 11:14:13 PM


In [93]:
USdf = df.query('iso2 == "US" and Case_Type == "Confirmed" and Province_State == "New York" and Combined_Key == "New York, New York, US"')

USdf['Date'] = pd.to_datetime(df.Date)
USdf = USdf.sort_values(by='Date')
USdf.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Case_Type,People_Total_Tested_Count,Cases,Difference,Date,Combined_Key,Country_Region,Province_State,Admin2,iso2,iso3,FIPS,Lat,Long,Population_Count,People_Hospitalized_Cumulative_Count,Data_Source,Prep_Flow_Runtime
776278,Confirmed,NaN,0,0,2020-01-22,"New York, New York, US",US,New York,New York,US,USA,36061.0,40.767273,-73.971526,8336817.0,NaN,2019 Novel Coronavirus COVID-19 (2019-nCoV) Da...,5/16/2020 11:14:13 PM
112769,Confirmed,NaN,0,0,2020-01-23,"New York, New York, US",US,New York,New York,US,USA,36061.0,40.767273,-73.971526,8336817.0,NaN,2019 Novel Coronavirus COVID-19 (2019-nCoV) Da...,5/16/2020 11:14:13 PM
440060,Confirmed,NaN,0,0,2020-01-24,"New York, New York, US",US,New York,New York,US,USA,36061.0,40.767273,-73.971526,8336817.0,NaN,2019 Novel Coronavirus COVID-19 (2019-nCoV) Da...,5/16/2020 11:14:13 PM
392390,Confirmed,NaN,0,0,2020-01-25,"New York, New York, US",US,New York,New York,US,USA,36061.0,40.767273,-73.971526,8336817.0,NaN,2019 Novel Coronavirus COVID-19 (2019-nCoV) Da...,5/16/2020 11:14:13 PM
343345,Confirmed,NaN,0,0,2020-01-26,"New York, New York, US",US,New York,New York,US,USA,36061.0,40.767273,-73.971526,8336817.0,NaN,2019 Novel Coronavirus COVID-19 (2019-nCoV) Da...,5/16/2020 11:14:13 PM


In [94]:
USdf.to_csv('NYCConfirmed.csv')

In [95]:
data = []
for i in USdf['Cases']:
    data.append([i])
data = [data[i:i+5] for i in range(0,len(data),5)]
data.remove(data[-1])
data=np.array(data)

#print(data)

predictions = []
counter=1
for i in USdf['Cases']:
    if counter>1 and counter%5==1:
        predictions.append([[i]])
    counter+=1
predictions=np.array(predictions)



#test/train split
x_train, y_train, x_test, y_test = data[:len(data)-5].astype('float32'), predictions[:len(data)-5].astype('float32'), data[len(data)-5:].astype('float32'), predictions[len(data)-5:].astype('float32')

#x_test = np.delete(x_test, x_test[-1])

print(data)
print(predictions)

[[[     0]
  [     0]
  [     0]
  [     0]
  [     0]]

 [[     0]
  [     0]
  [     0]
  [     0]
  [     0]]

 [[     0]
  [     0]
  [     0]
  [     0]
  [     0]]

 [[     0]
  [     0]
  [     0]
  [     0]
  [     0]]

 [[     0]
  [     0]
  [     0]
  [     0]
  [     0]]

 [[     0]
  [     0]
  [     0]
  [     0]
  [     0]]

 [[     0]
  [     0]
  [     0]
  [     0]
  [     0]]

 [[     0]
  [     0]
  [     0]
  [     0]
  [     0]]

 [[     1]
  [     1]
  [     1]
  [     4]
  [    11]]

 [[    11]
  [    12]
  [    19]
  [    25]
  [    55]]

 [[    95]
  [   154]
  [   269]
  [   269]
  [   463]]

 [[   814]
  [  1871]
  [  3954]
  [  5151]
  [  7530]]

 [[  9654]
  [ 12305]
  [ 14904]
  [ 17856]
  [ 21873]]

 [[ 25573]
  [ 29776]
  [ 33768]
  [ 37453]
  [ 43119]]

 [[ 47439]
  [ 51809]
  [ 57159]
  [ 63306]
  [ 67551]]

 [[ 72181]
  [ 76876]
  [ 81803]
  [ 87028]
  [ 92384]]

 [[ 98308]
  [103208]
  [106763]
  [110465]
  [118302]]

 [[123146]
  [127352]
  [131334

# Model

In [96]:
num_samples, time_steps, input_dim, output_dim = len(data), 5, 1, 1

model = NBeatsNet(backcast_length=time_steps, forecast_length=output_dim,
                  stack_types=(NBeatsNet.GENERIC_BLOCK, NBeatsNet.GENERIC_BLOCK,), nb_blocks_per_stack=4,
                  thetas_dim=(4, 4), share_weights_in_stack=True, hidden_layer_units=128)


model.compile_model(loss='mae', learning_rate=1e-5)

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=250, batch_size=7)

model.save('n_beats_model.nycconfirmed')

model2 = NBeatsNet.load('n_beats_model.nycconfirmed')




Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_variable (InputLayer)     (None, 5, 1)         0                                            
__________________________________________________________________________________________________
lambda_13 (Lambda)              (None, 5)            0           input_variable[0][0]             
__________________________________________________________________________________________________
0/0/generic/d1 (Dense)          (None, 128)          768         lambda_13[0][0]                  
                                                                 subtract_105[0][0]               
                                                                 subtract_106[0][0]               
                                                                 subtract_107[0][0]        

Train on 18 samples, validate on 5 samples
Epoch 1/250
18/18 [==============================] - 1s 67ms/step - loss: 28129.2358 - val_loss: 169093.0000
Epoch 2/250
18/18 [==============================] - 0s 1ms/step - loss: 27535.9375 - val_loss: 165285.2812
Epoch 3/250
18/18 [==============================] - 0s 1ms/step - loss: 26941.1123 - val_loss: 161407.6875
Epoch 4/250
18/18 [==============================] - 0s 1ms/step - loss: 26314.0116 - val_loss: 157547.7969
Epoch 5/250
18/18 [==============================] - 0s 981us/step - loss: 25849.6555 - val_loss: 153402.7500
Epoch 6/250
18/18 [==============================] - 0s 1000us/step - loss: 25181.2678 - val_loss: 149352.4531
Epoch 7/250
18/18 [==============================] - 0s 1ms/step - loss: 24658.0713 - val_loss: 145132.0312
Epoch 8/250
18/18 [==============================] - 0s 1ms/step - loss: 24002.7619 - val_loss: 141196.1719
Epoch 9/250
18/18 [==============================] - 0s 1ms/step - loss: 23493.5171 - v

18/18 [==============================] - 0s 1ms/step - loss: 1948.4609 - val_loss: 15307.1592
Epoch 77/250
18/18 [==============================] - 0s 1ms/step - loss: 1929.4235 - val_loss: 14770.2715
Epoch 78/250
18/18 [==============================] - 0s 976us/step - loss: 1930.4589 - val_loss: 14056.5527
Epoch 79/250
18/18 [==============================] - 0s 1ms/step - loss: 1947.9840 - val_loss: 13666.2373
Epoch 80/250
18/18 [==============================] - 0s 1ms/step - loss: 1941.5531 - val_loss: 13822.6592
Epoch 81/250
18/18 [==============================] - 0s 951us/step - loss: 1902.8163 - val_loss: 14543.8379
Epoch 82/250
18/18 [==============================] - 0s 926us/step - loss: 1836.5126 - val_loss: 15574.2910
Epoch 83/250
18/18 [==============================] - 0s 973us/step - loss: 1834.7586 - val_loss: 16343.5127
Epoch 84/250
18/18 [==============================] - 0s 1ms/step - loss: 1835.8029 - val_loss: 16689.7871
Epoch 85/250
18/18 [======================

18/18 [==============================] - 0s 1ms/step - loss: 465.9543 - val_loss: 4608.6123
Epoch 153/250
18/18 [==============================] - 0s 1ms/step - loss: 457.5505 - val_loss: 3739.0220
Epoch 154/250
18/18 [==============================] - 0s 1ms/step - loss: 459.3288 - val_loss: 3003.5032
Epoch 155/250
18/18 [==============================] - 0s 1ms/step - loss: 472.5134 - val_loss: 2661.0312
Epoch 156/250
18/18 [==============================] - 0s 1ms/step - loss: 444.8605 - val_loss: 3580.7937
Epoch 157/250
18/18 [==============================] - 0s 1ms/step - loss: 437.2880 - val_loss: 4236.0439
Epoch 158/250
18/18 [==============================] - 0s 1ms/step - loss: 453.0752 - val_loss: 3922.6438
Epoch 159/250
18/18 [==============================] - 0s 1ms/step - loss: 440.1303 - val_loss: 4571.4219
Epoch 160/250
18/18 [==============================] - 0s 1ms/step - loss: 443.2054 - val_loss: 4130.8218
Epoch 161/250
18/18 [==============================] - 0s 1m

Epoch 229/250
18/18 [==============================] - 0s 1ms/step - loss: 250.5411 - val_loss: 1589.7031
Epoch 230/250
18/18 [==============================] - 0s 886us/step - loss: 233.0394 - val_loss: 1393.8750
Epoch 231/250
18/18 [==============================] - 0s 892us/step - loss: 250.8381 - val_loss: 1370.8344
Epoch 232/250
18/18 [==============================] - 0s 926us/step - loss: 247.5526 - val_loss: 1004.6937
Epoch 233/250
18/18 [==============================] - 0s 964us/step - loss: 265.2879 - val_loss: 963.0219
Epoch 234/250
18/18 [==============================] - 0s 899us/step - loss: 222.7257 - val_loss: 1334.5687
Epoch 235/250
18/18 [==============================] - 0s 917us/step - loss: 252.6158 - val_loss: 2004.9094
Epoch 236/250
18/18 [==============================] - 0s 926us/step - loss: 256.4313 - val_loss: 1987.6406
Epoch 237/250
18/18 [==============================] - 0s 958us/step - loss: 249.6873 - val_loss: 1799.1813
Epoch 238/250
18/18 [==========

In [97]:
test = np.array([[[180000], [182216.], [184783.], [186289.], [189417.]]]).astype('float32')

predictions2 = model2.predict(test)

print(predictions2)



[[[191685.11]]]


# Now, let's predict COVID Deaths in NYC For 3 Days in Advance

**Normalization of the data here seems to make the prediction accuracy significantly worse...**

In [111]:
from sklearn import preprocessing
NYCdeaths = df.query('iso2 == "US" and Case_Type == "Deaths" and Province_State == "New York" and Combined_Key == "New York, New York, US"')

NYCdeaths['Date'] = pd.to_datetime(df.Date)
NYCdeaths = NYCdeaths.sort_values(by='Date')

#NYCdeaths['Cases'] = preprocessing.scale(NYCdeaths['Cases'].values)

NYCdeaths.to_csv('NYCdeaths.csv')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [115]:
data = []
for i in NYCdeaths['Cases']:
    data.append([i])
        
data = [data[i:i+5] for i in range(0,len(data),5)]

data.remove(data[-1])
data.remove(data[-1])
data=np.array(data)

predictions = []
counter=1
for i in NYCdeaths['Cases']:
    if counter>3 and counter%5==3:
        predictions.append([[i]])
    counter+=1
predictions=np.array(predictions)

print(predictions[-1])
#train/test split
x_train, y_train, x_test, y_test = data[:len(data)-5].astype('float32'), predictions[:len(data)-5].astype('float32'), data[len(data)-5:].astype('float32'), predictions[len(data)-5:].astype('float32')

[[20316]]


# Model

In [127]:
num_samples, time_steps, input_dim, output_dim = len(data), 5, 1, 1

model = NBeatsNet(backcast_length=time_steps, forecast_length=output_dim,
                  stack_types=(NBeatsNet.GENERIC_BLOCK, NBeatsNet.GENERIC_BLOCK,), nb_blocks_per_stack=8,
                  thetas_dim=(4, 4), share_weights_in_stack=True, hidden_layer_units=256)

model.compile_model(loss='mae', learning_rate=1e-5)

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=38, batch_size=3)

model.save('n_beats_model.nycdeaths')


Model: "model_25"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_variable (InputLayer)     (None, 5, 1)         0                                            
__________________________________________________________________________________________________
lambda_25 (Lambda)              (None, 5)            0           input_variable[0][0]             
__________________________________________________________________________________________________
0/0/generic/d1 (Dense)          (None, 256)          1536        lambda_25[0][0]                  
                                                                 subtract_281[0][0]               
                                                                 subtract_282[0][0]               
                                                                 subtract_283[0][0]        

Train on 17 samples, validate on 5 samples
Epoch 1/38
17/17 [==============================] - 2s 121ms/step - loss: 2204.7559 - val_loss: 15518.5869
Epoch 2/38
17/17 [==============================] - 0s 6ms/step - loss: 1887.5829 - val_loss: 13106.7373
Epoch 3/38
17/17 [==============================] - 0s 6ms/step - loss: 1658.1216 - val_loss: 10452.6428
Epoch 4/38
17/17 [==============================] - 0s 6ms/step - loss: 1403.4731 - val_loss: 7812.2723
Epoch 5/38
17/17 [==============================] - 0s 6ms/step - loss: 1124.2265 - val_loss: 5117.5790
Epoch 6/38
17/17 [==============================] - 0s 7ms/step - loss: 835.7923 - val_loss: 2251.3582
Epoch 7/38
17/17 [==============================] - 0s 7ms/step - loss: 583.1992 - val_loss: 973.5996
Epoch 8/38
17/17 [==============================] - 0s 7ms/step - loss: 280.1538 - val_loss: 4326.0063
Epoch 9/38
17/17 [==============================] - 0s 7ms/step - loss: 366.5232 - val_loss: 5658.8335
Epoch 10/38
17/17 [==

In [128]:
model2 = NBeatsNet.load('n_beats_model.nycdeaths')

test = np.array([data[-1]]).astype('float32')

predictions2 = model2.predict(test)

print(predictions2)

[[[19504.115]]]


## We can now use the above methods to predict any region, we just have to change the region of data we are scraping.

## Furthermore, we have tested predicting one and 3 days in advance, now let's try to predict 2 weeks in advance for the entire USA and see if our results are still good.

In [135]:
USAconf = df.query('iso3 == "USA" and Case_Type == "Confirmed"')

USAconf['Date'] = pd.to_datetime(df.Date)
USAconf = USAconf.sort_values(by='Date')

USAconf.to_csv('USAconfirmed.csv')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [143]:
ag_USAconf = USAconf.groupby(USAconf['Date'].dt.date).sum()
ag_USAconf.to_csv('aggregatedUSAconfirmed.csv')

## We will take 2 week windows of data to predict 2 weeks into the future

# preprocessing

In [260]:
data=[]
#history = deque(maxlen=14)

for i in range(len(ag_USAconf['Cases'])):
    if i+14 > len(ag_USAconf['Cases']):
        data.append([[ag_USAconf['Cases'][j]] for j in range (i,len(ag_USAconf['Cases']))])
    else:
        data.append([[ag_USAconf['Cases'][j]] for j in range (i,i+14)])

while len(data[-1]) < 14:
    data.remove(data[-1])

predictions=[]
for i in range(27,len(ag_USAconf['Cases'])):
    predictions.append([[ag_USAconf['Cases'][i]]])


data=np.array(data)
predictions=np.array(predictions)

future = np.array([data[i] for i in range(len(data)-14, len(data))]).astype('float32')


#delete last 14 samples from data that we stored in future since those predictions will be future predictions
for i in range(0,14):
    data = np.delete(data, -1,0)
    

# test/train split
splitter = int(0.9*len(data))
x_train, y_train, x_test, y_test = data[:splitter].astype('float32'), predictions[:splitter].astype('float32'), data[splitter:].astype('float32'), predictions[splitter:].astype('float32')

# Model

In [263]:
num_samples, time_steps, input_dim, output_dim = len(data), 14, 1, 1

model = NBeatsNet(backcast_length=time_steps, forecast_length=output_dim,
                  stack_types=(NBeatsNet.GENERIC_BLOCK, NBeatsNet.GENERIC_BLOCK,), nb_blocks_per_stack=16,
                  thetas_dim=(4, 4), share_weights_in_stack=True, hidden_layer_units=256)

model.compile_model(loss='mae', learning_rate=1e-5)

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=200, batch_size=5)

model.save('n_beats_model.usaconf')

Model: "model_43"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_variable (InputLayer)     (None, 14, 1)        0                                            
__________________________________________________________________________________________________
lambda_43 (Lambda)              (None, 14)           0           input_variable[0][0]             
__________________________________________________________________________________________________
0/0/generic/d1 (Dense)          (None, 256)          3840        lambda_43[0][0]                  
                                                                 subtract_609[0][0]               
                                                                 subtract_610[0][0]               
                                                                 subtract_611[0][0]        

Train on 80 samples, validate on 9 samples
Epoch 1/200
80/80 [==============================] - 5s 58ms/step - loss: 101157.0703 - val_loss: 297236.2622
Epoch 2/200
80/80 [==============================] - 0s 6ms/step - loss: 74828.4368 - val_loss: 43680.1942
Epoch 3/200
80/80 [==============================] - 0s 5ms/step - loss: 68997.0995 - val_loss: 12258.7778
Epoch 4/200
80/80 [==============================] - 1s 6ms/step - loss: 56921.7488 - val_loss: 13029.2357
Epoch 5/200
80/80 [==============================] - 1s 7ms/step - loss: 43374.5987 - val_loss: 67485.9158
Epoch 6/200
80/80 [==============================] - 1s 6ms/step - loss: 43724.2281 - val_loss: 58181.0408
Epoch 7/200
80/80 [==============================] - 0s 6ms/step - loss: 35863.2726 - val_loss: 134537.2483
Epoch 8/200
80/80 [==============================] - 1s 7ms/step - loss: 35739.5149 - val_loss: 37527.5836
Epoch 9/200
80/80 [==============================] - 1s 7ms/step - loss: 34593.2835 - val_loss: 1

80/80 [==============================] - 0s 6ms/step - loss: 26537.6709 - val_loss: 83271.0004
Epoch 77/200
80/80 [==============================] - 0s 6ms/step - loss: 20890.5085 - val_loss: 62925.7504
Epoch 78/200
80/80 [==============================] - 0s 6ms/step - loss: 21827.1676 - val_loss: 43825.9026
Epoch 79/200
80/80 [==============================] - 0s 5ms/step - loss: 24698.3858 - val_loss: 35620.4865
Epoch 80/200
80/80 [==============================] - 0s 5ms/step - loss: 20774.9147 - val_loss: 124279.6545
Epoch 81/200
80/80 [==============================] - 0s 6ms/step - loss: 20164.3632 - val_loss: 25827.4996
Epoch 82/200
80/80 [==============================] - 1s 6ms/step - loss: 17457.1562 - val_loss: 93689.2517
Epoch 83/200
80/80 [==============================] - 0s 6ms/step - loss: 19310.8657 - val_loss: 32443.2914
Epoch 84/200
80/80 [==============================] - 0s 6ms/step - loss: 16891.6063 - val_loss: 52173.7083
Epoch 85/200
80/80 [====================

80/80 [==============================] - 0s 5ms/step - loss: 18943.5151 - val_loss: 47457.9722
Epoch 152/200
80/80 [==============================] - 0s 5ms/step - loss: 21426.1049 - val_loss: 78739.1949
Epoch 153/200
80/80 [==============================] - 0s 5ms/step - loss: 18523.1900 - val_loss: 51923.2365
Epoch 154/200
80/80 [==============================] - 0s 5ms/step - loss: 18378.0076 - val_loss: 71778.0977
Epoch 155/200
80/80 [==============================] - 0s 5ms/step - loss: 20755.3003 - val_loss: 52796.6111
Epoch 156/200
80/80 [==============================] - 1s 8ms/step - loss: 20133.5548 - val_loss: 38600.3331
Epoch 157/200
80/80 [==============================] - 1s 7ms/step - loss: 20315.7937 - val_loss: 119775.4436
Epoch 158/200
80/80 [==============================] - 0s 5ms/step - loss: 20714.0049 - val_loss: 36270.1806
Epoch 159/200
80/80 [==============================] - 0s 6ms/step - loss: 20378.8523 - val_loss: 79439.1523
Epoch 160/200
80/80 [===========

In [264]:
model2 = NBeatsNet.load('n_beats_model.usaconf')

predictions2 = model2.predict(future)

print(predictions2)

[[[1556676.2]]

 [[1589130.6]]

 [[1631866. ]]

 [[1679314.5]]

 [[1726944.8]]

 [[1769211.4]]

 [[1798818.8]]

 [[1830017.5]]

 [[1864582. ]]

 [[1908317.2]]

 [[1955048.8]]

 [[2002468.2]]

 [[2043073.9]]

 [[2077294. ]]]


# Now, let's model USA Deaths, forecasting 1 week in advance

In [255]:
USAdeaths = df.query('iso3 == "USA" and Case_Type == "Deaths"')

USAdeaths['Date'] = pd.to_datetime(df.Date)
USAdeaths = USAdeaths.sort_values(by='Date')

USAdeaths.to_csv('USAdeaths.csv')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [256]:
ag_USAdeaths = USAdeaths.groupby(USAdeaths['Date'].dt.date).sum()
ag_USAdeaths.to_csv('aggregatedUSAdeaths.csv')

In [273]:
data=[]
for i in range(len(ag_USAdeaths['Cases'])):
    if i+14 > len(ag_USAdeaths['Cases']):
        data.append([[ag_USAdeaths['Cases'][j]] for j in range (i,len(ag_USAdeaths['Cases']))])
    else:
        data.append([[ag_USAdeaths['Cases'][j]] for j in range (i,i+14)])
        
while len(data[-1]) < 14:
    data.remove(data[-1])

predictions=[]
for i in range(20,len(ag_USAdeaths['Cases'])):
    predictions.append([[ag_USAdeaths['Cases'][i]]])
    
data=np.array(data)
predictions=np.array(predictions)

future = np.array([data[i] for i in range(len(data)-7, len(data))]).astype('float32')

for i in range(0,7):
    data = np.delete(data, -1,0)
    
splitter = int(0.9*len(data))
x_train, y_train, x_test, y_test = data[:splitter].astype('float32'), predictions[:splitter].astype('float32'), data[splitter:].astype('float32'), predictions[splitter:].astype('float32')

print(data)
print(predictions)

[[[    0]
  [    0]
  [    0]
  ...
  [    0]
  [    0]
  [    0]]

 [[    0]
  [    0]
  [    0]
  ...
  [    0]
  [    0]
  [    0]]

 [[    0]
  [    0]
  [    0]
  ...
  [    0]
  [    0]
  [    0]]

 ...

 [[51403]
  [53659]
  [54783]
  ...
  [70951]
  [73342]
  [75546]]

 [[53659]
  [54783]
  [56121]
  ...
  [73342]
  [75546]
  [77059]]

 [[54783]
  [56121]
  [58255]
  ...
  [75546]
  [77059]
  [78673]]]
[[[    0]]

 [[    0]]

 [[    0]]

 [[    0]]

 [[    0]]

 [[    0]]

 [[    0]]

 [[    0]]

 [[    0]]

 [[    0]]

 [[    0]]

 [[    0]]

 [[    0]]

 [[    0]]

 [[    0]]

 [[    0]]

 [[    0]]

 [[    0]]

 [[    1]]

 [[    1]]

 [[    6]]

 [[    7]]

 [[   11]]

 [[   12]]

 [[   14]]

 [[   17]]

 [[   21]]

 [[   22]]

 [[   28]]

 [[   32]]

 [[   41]]

 [[   50]]

 [[   56]]

 [[   70]]

 [[   98]]

 [[  132]]

 [[  189]]

 [[  266]]

 [[  362]]

 [[  451]]

 [[  593]]

 [[  777]]

 [[ 1004]]

 [[ 1312]]

 [[ 1722]]

 [[ 2258]]

 [[ 2879]]

 [[ 3511]]

 [[ 4306]]

# Model

In [282]:
num_samples, time_steps, input_dim, output_dim = len(data), 14, 1, 1

model = NBeatsNet(backcast_length=time_steps, forecast_length=output_dim,
                  stack_types=(NBeatsNet.GENERIC_BLOCK, NBeatsNet.GENERIC_BLOCK,), nb_blocks_per_stack=12,
                  thetas_dim=(4, 4), share_weights_in_stack=True, hidden_layer_units=256)


model.compile_model(loss='mae', learning_rate=1e-5)

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=55, batch_size=5)

model.save('n_beats_model.usadeaths')

Model: "model_49"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_variable (InputLayer)     (None, 14, 1)        0                                            
__________________________________________________________________________________________________
lambda_49 (Lambda)              (None, 14)           0           input_variable[0][0]             
__________________________________________________________________________________________________
0/0/generic/d1 (Dense)          (None, 256)          3840        lambda_49[0][0]                  
                                                                 subtract_689[0][0]               
                                                                 subtract_690[0][0]               
                                                                 subtract_691[0][0]        

Train on 86 samples, validate on 10 samples
Epoch 1/55
86/86 [==============================] - 4s 42ms/step - loss: 7606.0228 - val_loss: 24434.8574
Epoch 2/55
86/86 [==============================] - 1s 6ms/step - loss: 3293.1761 - val_loss: 13792.0122
Epoch 3/55
86/86 [==============================] - 0s 5ms/step - loss: 1942.6887 - val_loss: 14423.3477
Epoch 4/55
86/86 [==============================] - 0s 5ms/step - loss: 1786.7794 - val_loss: 8879.7661
Epoch 5/55
86/86 [==============================] - 1s 7ms/step - loss: 1350.3764 - val_loss: 8752.2063
Epoch 6/55
86/86 [==============================] - 1s 7ms/step - loss: 1213.2063 - val_loss: 5018.5994
Epoch 7/55
86/86 [==============================] - 0s 5ms/step - loss: 1094.0130 - val_loss: 3186.7374
Epoch 8/55
86/86 [==============================] - 0s 5ms/step - loss: 1026.9033 - val_loss: 6056.9639
Epoch 9/55
86/86 [==============================] - 1s 6ms/step - loss: 1251.5025 - val_loss: 1503.1391
Epoch 10/55
86/8

In [283]:
model2 = NBeatsNet.load('n_beats_model.usadeaths')

predictions2 = model2.predict(future)

print(predictions2)

[[[ 95481.23]]

 [[ 97326.13]]

 [[ 98810.43]]

 [[100092.39]]

 [[101553.2 ]]

 [[102520.71]]

 [[103796.99]]]
